<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [5]</a>'.</span>

Author: Nicolas Legrand <nicolas.legrand@cfin.au.dk>

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns
import numpy as np
from metadPy import sdt
from metadPy.utils import trials2counts, discreteRatings
from metadPy.plotting import plot_confidence
from systole.detection import oxi_peaks

sns.set_context('talk')
%matplotlib inline

# Heart Rate Discrimination task - Summary results

This notebook introduces basic analysis steps, plots and quality check for the **Heart Rate Discrimination task**. The current version use data from a young and healthy participant tested with the default task parameters implemented in the `launcher.py` file (80 trials per condition, 30 using a 1-Up/1-Down staircase and 50 using the Psi method.

The `path` variable can be adapted to load new data. The target directory should include the following files output by the task: `final.txt`, containing the behavioural data, `Intero_posterior.npy` and `Extero_posterior.npy`, containing the posterior estimates and `signal.txt` for the PPG signal time series.

You can also use this notebook as a template to generate individual reports of groups of participants (see `reports.py`).

For group-level summary statistics, please refer to `HRDgroupLevel.ipynb`.

**Import data**

In [2]:
path = 'C:/Users/au646069/ECG/1_VPN_aux/'
subject = 'sub_0019'

In [3]:
# Parameters
subject = "sub_0248"
path = "C:/Users/au646069/ECG/1_VPN_aux/"


In [4]:
resultsFiles = os.listdir(os.path.join(path, subject, 'HRD'))

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [5]:
# Logs dataframe
df = pd.read_csv(os.path.join(path, subject, 'HRD', [file for file in resultsFiles if file.endswith('final.txt')][0]))

# History of posteriors distribution
interoPost = np.load(os.path.join(path, subject, 'HRD', [file for file in resultsFiles if file.endswith('Intero_posterior.npy')][0]))
exteroPost = np.load(os.path.join(path, subject, 'HRD', [file for file in resultsFiles if file.endswith('Extero_posterior.npy')][0]))

# PPG signal
signal_df = pd.read_csv(os.path.join(path, subject, 'HRD', [file for file in resultsFiles if file.endswith('signal.txt')][0]))
signal_df['Time'] = np.arange(0, len(signal_df))/1000 # Create time vector

ValueError: cannot reshape array of size 0 into shape (80,82,200)

In [ ]:
df = df[df.RatingProvided == 1]
df['DecisionRT'] = df['EstimationRT']
df['Decision'] = df['Estimation']
df['TrialType'] = df['StairCond']

In [ ]:
this_df = df[df.Modality=='Intero']
sum(this_df.StairCond == 'psi'), sum(this_df.StairCond == 'psiCatchTrial'), sum(this_df.StairCond == 'low'), sum(this_df.StairCond == 'high')

# Response time

In [ ]:
palette = ['#b55d60', '#5f9e6e']

fig, axs = plt.subplots(1, 2, figsize=(13, 5))
for i, task, title in zip([0, 1], ['DecisionRT', 'ConfidenceRT'], ['Decision', 'Confidence']):
    sns.boxplot(data=df, x='Modality', y='DecisionRT', hue='ResponseCorrect',
                palette=palette, width=.15, notch=True, ax=axs[i])
    sns.stripplot(data=df, x='Modality', y='DecisionRT', hue='ResponseCorrect',
                  dodge=True, linewidth=1, size=6, palette=palette, alpha=.6, ax=axs[i])
    axs[i].set_title(title)
    axs[i].set_ylabel('Response Time (s)')
    axs[i].set_xlabel('')
    axs[i].get_legend().remove()
sns.despine(trim=10)

handles, labels = axs[0].get_legend_handles_labels()
plt.legend(handles[0:2], ['Incorrect', 'Correct'], bbox_to_anchor=(1.05, .5), loc=2, borderaxespad=0.)

Response time distribution for the decision and the confidence rating phases for correct (red) and incorrect (green) responses.

# Metacognition

SDT estimate for decision 1 perforamces (d' and criterion)

In [ ]:
for i, cond in enumerate(['Intero', 'Extero']):
    this_df = df[df.Modality == cond].copy()
    this_df['signal'] = (this_df.responseBPM > this_df.listenBPM)
    this_df['responses'] = (this_df.Decision == 'More')

    hit, miss, fa, cr = sdt.scores(data=this_df, stimuli='signal')
    hr, far = sdt.rates(hits=hit, misses=miss, fas=fa, crs=cr)
    d, c = sdt.dprime(hit_rate=hr, fa_rate=far), sdt.criterion(hit_rate=hr, fa_rate=far)
    
    print(f'Condition: {cond} - d-prime: {d} - criterion: {c}')

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(13, 5))

for i, cond in enumerate(['Intero', 'Extero']):
    try:
        this_df = df[df.Modality == cond]
        this_df = this_df[~this_df.Confidence.isnull()]
        new_confidence, _ = discreteRatings(this_df.Confidence)
        this_df['Confidence'] = new_confidence
        this_df['Stimuli'] = (this_df.Alpha > 0).astype('int')
        this_df['Responses'] = (this_df.Decision == 'More').astype('int')
        nR_S1, nR_S2 = trials2counts(data=this_df)
        plot_confidence(nR_S1, nR_S2, ax=axs[i])
        axs[i].set_title(f'{cond} condition')
    except:
        print('Bab bins')
sns.despine()
plt.tight_layout()

Distribution of confidence ratings for correct (green) and incorrect (red) trials. Overlapping distribution suggests that the subjective confidence in the decision was not predictive of decision performances.

# Psychophysics

Distribution of the intensities values tested by each staircases (Intero and Extero).

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(8, 5))

for cond, col in zip(['Intero', 'Extero'], ['#c44e52', '#4c72b0']):
    this_df = df[df.Modality == cond]
    axs.hist(this_df.Alpha, color=col, bins=np.arange(-40.5, 40.5, 5), histtype='stepfilled',
             ec="k", density=True, align='mid', label=cond, alpha=.6)
axs.set_title('Distribution of the tested intensities values')
axs.set_xlabel('Intensity (BPM)')
plt.legend()
sns.despine(trim=10)
plt.tight_layout()

## Threshold

### Up/Down stairecases

In [ ]:
if sum(this_df.StairCond == 'UpDown') > 0:

    fig, axs = plt.subplots(figsize=(15, 5), nrows=1, ncols=2)

    # Staircase traces
    for i, cond, col in zip([0, 1], ['Intero', 'Extero'], ['#c44e52', '#4c72b0']):
        this_df = df[(df.Modality == cond) & (df.TrialType == 'UpDown')]

        axs[i].plot(np.arange(0, len(this_df))[this_df.StairCond == 'high'], 
                        this_df.Alpha[this_df.StairCond == 'high'], linestyle='--', color=col, linewidth=2)
        axs[i].plot(np.arange(0, len(this_df))[this_df.StairCond == 'low'], 
                        this_df.Alpha[this_df.StairCond == 'low'], linestyle='-', color=col, linewidth=2)

        axs[i].plot(np.arange(0, len(this_df))[this_df.Decision == 'More'], 
                        this_df.Alpha[this_df.Decision == 'More'], col, marker='o', linestyle='', markeredgecolor='k', label=cond)
        axs[i].plot(np.arange(0, len(this_df))[this_df.Decision == 'Less'], 
                        this_df.Alpha[this_df.Decision == 'Less'], 'w', marker='s', linestyle='', markeredgecolor=col, label=cond)

        axs[i].axhline(y=0, linestyle='--', color = 'gray')
        handles, labels = axs[i].get_legend_handles_labels()
        axs[i].legend(handles[0:2], ['More', 'Less'], borderaxespad=0., title='Decision')
        axs[i].set_ylabel('Intensity ($\Delta$ BPM)')
        axs[i].set_xlabel('Trials')
        axs[i].set_ylim(-42, 42)
        axs[i].set_title(cond+'ception')
        sns.despine(trim=10, ax=axs[i])
        plt.gcf()

### Psi

In [ ]:
if sum(this_df.TrialType == 'psi') > 0:

    fig, axs = plt.subplots(figsize=(15, 5), nrows=1, ncols=2)

    # Plot confidence interval for each staircase
    def ci(x):
        return np.where(np.cumsum(x) / np.sum(x) > .025)[0][0], \
               np.where(np.cumsum(x) / np.sum(x) < .975)[0][-1]

    for i, stair, col in zip([0, 1], [interoPost, exteroPost], ['#c44e52', '#4c72b0']):
        ciUp, ciLow = [], []
        for t in range(stair.shape[0]):
            up, low = ci(stair.mean(2)[t])
            ciUp.append(np.arange(-40.5, 40.5)[up])
            ciLow.append(np.arange(-40.5, 40.5)[low])
        axs[i].fill_between(x=np.arange(len(ciLow)),
                            y1=ciLow,
                            y2=ciUp,
                            color=col, alpha=.2)

    # Staircase traces
    for i, cond, col in zip([0, 1], ['Intero', 'Extero'], ['#c44e52', '#4c72b0']):
        this_df = df[(df.Modality == cond) & (df.TrialType != 'UpDown')]

        # Show UpDown staircase traces
        axs[i].plot(np.arange(0, len(this_df))[this_df.StairCond == 'high'], 
                        this_df.Alpha[this_df.StairCond == 'high'], linestyle='--', color=col, linewidth=2)
        axs[i].plot(np.arange(0, len(this_df))[this_df.StairCond == 'low'], 
                        this_df.Alpha[this_df.StairCond == 'low'], linestyle='-', color=col, linewidth=2)

        axs[i].plot(np.arange(0, len(this_df))[this_df.Decision == 'More'], 
                        this_df.Alpha[this_df.Decision == 'More'], col, marker='o', linestyle='', markeredgecolor='k', label=cond)
        axs[i].plot(np.arange(0, len(this_df))[this_df.Decision == 'Less'], 
                        this_df.Alpha[this_df.Decision == 'Less'], 'w', marker='s', linestyle='', markeredgecolor=col, label=cond)

        # Trheshold estimate
        axs[i].plot(np.arange(sum(this_df.StairCond != 'psi'), len(this_df)), this_df[this_df.StairCond == 'psi'].EstimatedThreshold, linestyle='-', color=col, linewidth=4)
        axs[i].plot(np.arange(0, sum(this_df.StairCond != 'psi')), this_df[this_df.StairCond != 'psi'].EstimatedThreshold, linestyle='--', color=col, linewidth=2, alpha=.3)

        axs[i].axhline(y=0, linestyle='--', color = 'gray')
        handles, labels = axs[i].get_legend_handles_labels()
        axs[i].legend(handles[0:2], ['More', 'Less'], borderaxespad=0., title='Decision')
        axs[i].set_ylabel('Intensity ($\Delta$ BPM)')
        axs[i].set_xlabel('Trials')
        axs[i].set_ylim(-42, 42)
        axs[i].set_title(cond+'ception')
        sns.despine(trim=10, ax=axs[i])
        plt.gcf()

This figure represents the evolution of threshold estimate across trials for the Interoception and Exteroception condition. Shaded areas represent the 95% confidence interval of the threshold estimate by Psi. For each condition, the first 30 trials (connected with dashed lines) were allocated to an Up/Down method (2 interleaved staircases starting a -40.5 or 40 respectively). The intensities and responses were included in the Psi staircase to maximize the amount of information included. The remaining 50 trials were monitored by the Psi staircase only. This dual estimation was implemented to estimate the reliability of the estimation of threshold using an up/down procedure, as compared to a longer psi procedure.

## Psychometric function

In [ ]:
sns.set_context('talk')
fig, axs = plt.subplots(figsize=(8, 5))
for i, modality, col in zip((0, 1), ['Extero', 'Intero'], ['#4c72b0', '#c44e52']):
    
    this_df = df[(df.Modality == modality) & (df.TrialType == 'psi')]

    t, s = this_df.EstimatedThreshold.iloc[-1], this_df.EstimatedSlope.iloc[-1]
    # Plot Psi estimate of psychometric function
    axs.plot(np.linspace(-40, 40, 500), 
            (norm.cdf(np.linspace(-40, 40, 500), loc=t, scale=s)),
            '--', color=col, label=modality)
    # Plot threshold
    axs.plot([t, t], [0, .5], color=col, linewidth=2)
    axs.plot(t, .5, 'o', color=col, markersize=10)

    # Plot data points
    for ii, intensity in enumerate(np.sort(this_df.Alpha.unique())):
        resp = sum((this_df.Alpha == intensity) & (this_df.Decision == 'More'))
        total = sum(this_df.Alpha == intensity)
        axs.plot(intensity, resp/total, 'o', alpha=0.5, color=col, markeredgecolor='k', markersize=total*5)
plt.ylabel('P$_{(Response = More|Intensity)}$')
plt.xlabel('Intensity ($\Delta$ BPM)')
plt.tight_layout()
plt.legend()
sns.despine()

Psychometric curves fitted using the estimated threshod and slope from the psi staircase after 80 trials for each condition. We only keeped the value estimated in the last trial. The size of the circles reflect the proportion of trial for each intensity level.

# Pulse oximeter

## Visualization of PPG signal

This interactive graph show the PPG signal recorded at each interoceptive trial. Blue and red time series represent different trials of 6 seconds each. The 5 last seconds were used to estimate the average heart rate of the participant, the first second was included to help peak detection algorithm initialization.

In [ ]:
meanBPM, stdBPM, rangeBPM = [], [], []

fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(30, 10))
for i, trial in enumerate(signal_df.nTrial.unique()[:int(signal_df.nTrial.nunique()/2)]):
    color = '#c44e52' if (i % 2) == 0 else '#4c72b0'
    this_df = signal_df[signal_df.nTrial==trial]  # Downsample to save memory
    ax[0].plot(this_df.Time, this_df.signal, label='PPG', color=color, linewidth=.5)

    # Peaks detection
    signal, peaks = oxi_peaks(this_df.signal, sfreq=1000)
    bpm = 60000/np.diff(np.where(peaks[-5000:])[0])
    m, s, r = bpm.mean(), bpm.std(), bpm.max() - bpm.min()
    meanBPM.append(m)
    stdBPM.append(s)
    rangeBPM.append(r)
    
    # Plot instantaneous heart rate
    ax[1].plot(this_df.Time.to_numpy()[np.where(peaks)[0][1:]], 
               60000/np.diff(np.where(peaks)[0]),
              'o-', color='gray', alpha=0.6)
    
ax[1].set_xlabel("Time (s)")
ax[0].set_ylabel("PPG level (a.u.)")
ax[1].set_ylabel("Heart rate (BPM)")
ax[0].set_title("PPG signal recorded during interoceptive condition (5 seconds each)")
sns.despine()

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(30, 10))
for i, trial in enumerate(signal_df.nTrial.unique()[int(signal_df.nTrial.nunique()/2):]):
    color = '#c44e52' if (i % 2) == 0 else '#4c72b0'
    this_df = signal_df[signal_df.nTrial==trial]  # Downsample to save memory
    ax[0].plot(this_df.Time, this_df.signal, label='PPG', color=color, linewidth=.5)

    # Peaks detection
    signal, peaks = oxi_peaks(this_df.signal, sfreq=1000)
    bpm = 60000/np.diff(np.where(peaks[-5000:])[0])
    m, s, r = bpm.mean(), bpm.std(), bpm.max() - bpm.min()
    meanBPM.append(m)
    stdBPM.append(s)
    rangeBPM.append(r)
    
    # Plot instantaneous heart rate
    ax[1].plot(this_df.Time.to_numpy()[np.where(peaks)[0][1:]], 
               60000/np.diff(np.where(peaks)[0]),
              'o-', color='gray', alpha=0.6)
    
ax[1].set_xlabel("Time (s)")
ax[0].set_ylabel("PPG level (a.u.)")
ax[1].set_ylabel("Heart rate (BPM)")
ax[0].set_title("PPG signal recorded during interoceptive condition (5 seconds each)")
sns.despine()

## Heart rate - Summary statistics

This figure show the evolution of the average and standard deviation of the instantaneous heart rate across time. An instantaneous frequnecy was derived between each peak detected in the PPG signal (also known as pulse-to-pulse intervals, or pseudo RR intervals). Rapid increase or decrease of the heart rate frequency can lead to larger standard deviation, and less accurate estimation of the average heart rate.

In [ ]:
sns.set_context('talk')
fig, axs = plt.subplots(figsize=(13, 5), nrows=2, ncols=2)
for i, metric, col in zip(range(3), [meanBPM, stdBPM], ['#b55d60', '#5f9e6e']):
    axs[i, 0].plot(metric, 'o-', color=col, alpha=.6)
    axs[i, 1].hist(metric, color=col, bins=15, ec="k", density=True, alpha=.6)
    axs[i, 0].set_ylabel('Mean BPM' if i == 0 else 'STD BPM')
    axs[i, 0].set_xlabel('Trials')
    axs[i, 1].set_xlabel('BPM')
sns.despine()
plt.tight_layout()